In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["tensorwaves[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Using TensorWaves with AmpForm

(compwa-step-1)=
## Step 1: Create amplitude model

Whether {ref}`generating data <compwa-step-2>` or {ref}`fitting a model <compwa-step-3>`, TensorWaves takes mathematical expressions as input. When that expression is an amplitude model, it is most convenient to formulate it with {mod}`ampform`.

This notebook illustrates how to create such an amplitude model with {mod}`ampform` and how to write it to a recipe file that can be understood by {mod}`tensorwaves`. For more advanced examples, have a look at {doc}`the usage guides of AmpForm <ampform:usage>`.

In this example, we use the helicity formalism, but you can also use `formalism="canonical-helicity"`. As you can see, we analyze the decay $J/\psi \to \pi^0\pi^0\gamma$ here.

```{admonition} Simplified model: $J/\psi \to f_0\gamma$
---
class: dropdown
---
As {ref}`compwa-step-3` serves to illustrate usage only, we make the amplitude model here a bit simpler by not allowing $\omega$ resonances (which are narrow and therefore hard to fit). For this reason, we can also limit the {class}`~qrules.settings.InteractionType` to {attr}`~qrules.settings.InteractionType.STRONG`.
```

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="helicity",
)

As a small goodie, you can use [`graphviz`](https://pypi.org/project/graphviz) to {doc}`visualize <qrules:usage/visualize>` the generated graphs:

In [ ]:
from graphviz import Source

dot = qrules.io.asdot(reaction, collapse_graphs=True)
Source(dot)

Next we convert the {attr}`~qrules.transition.ReactionInfo.transitions` into an amplitude model (here: {class}`~ampform.helicity.HelicityModel`). This can be done with {func}`~ampform.get_builder` and {meth}`~ampform.helicity.HelicityAmplitudeBuilder.formulate`.

In [ ]:
import ampform

model_builder = ampform.get_builder(reaction)
model = model_builder.formulate()
display(*model.parameter_defaults)

The heart of the model is a sympy expression that contains the full description of the intensity model. Note two things:
1. The coefficients for the different amplitudes are **complex** valued.
2. By default there is no dynamics in the model, so it still has to be specified.

We choose to use {func}`~ampform.dynamics.relativistic_breit_wigner_with_ff` as the lineshape for all resonances and use a Blatt-Weisskopf form factor ({func}`~ampform.dynamics.builder.create_non_dynamic_with_ff`) for the production decay. The {meth}`~ampform.helicity.HelicityAmplitudeBuilder.set_dynamics` is a convenience interface for replacing the dynamics for intermediate states.

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.set_dynamics("J/psi(1S)", create_non_dynamic_with_ff)
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

Now let's take another look at the parameters of the model to see which new parameters are there:

In [ ]:
sorted(model.parameter_defaults, key=lambda s: s.name)

Finally, we can write the {class}`~ampform.helicity.HelicityModel` to disk via {mod}`pickle`. The {class}`~qrules.transition.ReactionInfo` object can be pickled as well, but here, we write it to JSON:

In [ ]:
import pickle

qrules.io.write(reaction, "transitions.json")
with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

Cool, that's it! We now have a template for an amplitude model with which to {ref}`generate data <compwa-step-2>` and {ref}`perform a fit <compwa-step-3>`. In the next steps, we will use use this {class}`~ampform.helicity.HelicityModel` as a fit model template for {mod}`tensorwaves`.

(compwa-step-2)=
## Step 2: Generate data

In this section, we will use the {class}`~ampform.helicity.HelicityModel` that we created with {mod}`ampform` in {ref}`the previous step <compwa-step-1>` to generate a data sample via hit & miss Monte Carlo. We do this with the {mod}`.data` module.

First, we {func}`~pickle.load` the {class}`~ampform.helicity.HelicityModel` that was created in the previous step:

In [ ]:
import pickle

from ampform.helicity import HelicityModel

with open("helicity_model.pickle", "rb") as model_file:
    model: HelicityModel = pickle.load(model_file)

In [ ]:
reaction_info = model.reaction_info
initial_state = next(iter(reaction_info.initial_state.values()))
print("Initial state:")
print(" ", initial_state.name)
print("Final state:")
for i, p in reaction_info.final_state.items():
    print(f"  {i}: {p.name}")

### 2.1 Generate phase space sample

The {class}`~qrules.transition.ReactionInfo` class defines the constraints of the phase space. As such, we have enough information to generate a **phase-space sample** for this particle reaction. We do this with the {func}`.generate_phsp` function. By default, this function uses {class}`.TFPhaseSpaceGenerator` as a, well... phase-space generator (using {obj}`tensorflow <tf.Tensor>` and the [`phasespace`](https://phasespace.readthedocs.io) package as a back-end) and generates random numbers with {class}`.TFUniformRealNumberGenerator`. You can use other generators with the arguments of {func}`.generate_phsp`.

As opposed to the main {ref}`usage:Generate data sample` of the main usage example page, we will generate a **deterministic** data sample. This can be done by feeding a {class}`.UniformRealNumberGenerator` with a specific {attr}`~.UniformRealNumberGenerator.seed` and feeding that generator to the {func}`.generate_phsp` function:

In [ ]:
import numpy as np
import pandas as pd

from tensorwaves.data import TFUniformRealNumberGenerator, generate_phsp

rng = TFUniformRealNumberGenerator(seed=0)
initial_state_mass = reaction_info.initial_state[-1].mass
final_state_masses = {i: p.mass for i, p in reaction_info.final_state.items()}
phsp_momenta = generate_phsp(
    size=100_000,
    initial_state_mass=initial_state_mass,
    final_state_masses=final_state_masses,
    random_generator=rng,
)
pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in phsp_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

The resulting phase space sample is a {obj}`dict` of final state IDs to an {obj}`~numpy.array` of four-momenta. In the last step, we converted this sample in such a way that it is rendered as an understandable {class}`pandas.DataFrame`.

### 2.2 Generate intensity-based sample

'Data samples' are more complicated than phase space samples in that they represent the intensity profile resulting from a reaction. You therefore need a {class}`.Function` object that expresses an intensity distribution as well as a phase space over which to generate that distribution. We call such a data sample an **intensity-based sample**.

An intensity-based sample is generated with the function {func}`.generate_data`. Its usage is similar to {func}`.generate_phsp`, but now you have to provide a {obj}`.Function` as well as a {obj}`.DataTransformer` that is used to transform the four-momentum phase space sample to a data sample that can be understood by the {obj}`.Function`.

Now, recall that in {ref}`compwa-step-1`, we used the helicity formalism to mathematically express the reaction in terms of an amplitude model. TensorWaves needs to convert this {obj}`~ampform.helicity.HelicityModel` to a {obj}`.Function` object that can perform fast computations. This can be done with {func}`.create_parametrized_function`:

:::{margin}

Here, we make use of {func}`.fast_lambdify` by specifying `max_complexity`. See {ref}`usage/faster-lambdify:Specifying complexity`.

:::

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

intensity = create_parametrized_function(
    expression=model.expression.doit(),
    parameters=model.parameter_defaults,
    max_complexity=200,
    backend="numpy",
)

A problem is that {class}`.ParametrizedBackendFunction` takes a {obj}`.DataSample` with kinematic variables for the helicity formalism as input, not a set of four-momenta. We therefore need to construct a {class}`.DataTransformer` to transform these four-momenta to function variables. In this case, we work with the helicity formalism, so we construct a {class}`.SympyDataTransformer`:

In [ ]:
from tensorwaves.data.transform import SympyDataTransformer

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)

That's it, now we have enough info to create an intensity-based data sample. Notice how the structure of the output data is the same as the {ref}`phase-space sample we generated previously <usage/ampform:2.1 Generate phase space sample>`:

In [ ]:
from tensorwaves.data import generate_data

data_momenta = generate_data(
    size=10_000,
    initial_state_mass=initial_state_mass,
    final_state_masses=final_state_masses,
    data_transformer=helicity_transformer,
    intensity=intensity,
    random_generator=rng,
)
pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in data_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

As before, we use a {class}`.UniformRealNumberGenerator` with a specific {attr}`~.UniformRealNumberGenerator.seed` to ensure we get a **deterministic** data sample.

### 2.3 Visualize kinematic variables

We now have a phase space sample and an intensity-based sample. Their data structure isn't the most informative though: it's just a collection of four-momentum tuples. But we can again use the {class}`.SympyDataTransformer` to convert these four-momenta to (in the case of the helicity formalism) invariant masses and helicity angles:

In [ ]:
phsp = helicity_transformer(phsp_momenta)
data = helicity_transformer(data_momenta)
list(data)

The {obj}`.DataSample` is a mapping of kinematic variables names to a 1-dimensional array of values. The numbers you see here are final state IDs as defined in the {class}`~ampform.helicity.HelicityModel` member of the {class}`~ampform.helicity.HelicityModel`:

In [ ]:
for state_id, particle in reaction_info.final_state.items():
    print(f"ID {state_id}:", particle.name)

````{admonition} Available kinematic variables
---
class: dropdown
---
By default, {mod}`tensorwaves` only generates invariant masses of the {class}`Topologies <qrules.topology.Topology>` that are of relevance to the decay problem. In this case, we only have resonances $f_0 \to \pi^0\pi^0$. If you are interested in more invariant mass combinations, you can do so with the method {meth}`~ampform.kinematics.HelicityAdapter.register_topology`.
````

The {obj}`.DataSample` can easily be converted to a {class}`pandas.DataFrame`:

In [ ]:
import pandas as pd

data_frame = pd.DataFrame(data)
phsp_frame = pd.DataFrame(data)
data_frame

This also means that we can use all kinds of fancy plotting functionality of for instance {mod}`matplotlib.pyplot` to see what's going on. Here's an example:

In [ ]:
from matplotlib import cm

resonances = sorted(
    reaction_info.get_intermediate_particles(),
    key=lambda p: p.mass,
)

evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [cm.rainbow(x) for x in evenly_spaced_interval]

In [ ]:
import matplotlib.pyplot as plt

data_frame["m_12"].hist(bins=100, alpha=0.5, density=True, figsize=(9, 4))
plt.xlabel("$m$ [GeV]")
for p, color in zip(resonances, colors):
    plt.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
plt.legend()
plt.show()

:::{seealso}

{ref}`usage/ampform:Intensity components`

:::

### 2.4 Export data sets

To export the generated data samples, simply {func}`pickle.dump` them as follows:

In [ ]:
import pickle

with open("data.pickle", "wb") as stream:
    pickle.dump(data, stream)
with open("phsp.pickle", "wb") as stream:
    pickle.dump(phsp, stream)

In the {ref}`next step <compwa-step-3>`, we illustrate how to {meth}`~.Minuit2.optimize` the intensity model to these data samples.

(compwa-step-3)=
## Step 3: Perform fit

As explained in the {ref}`previous step <compwa-step-2>`, a {class}`.ParametrizedFunction` can compute a list of intensities (real numbers) for an input {obj}`.DataSample`. At this stage, we want to optimize the parameters of this {class}`.ParametrizedFunction`, so that it matches the distribution of our data sample. This is what we call 'fitting'.

First, we load the relevant data from the previous steps. Notice that we use {func}`.create_parametrized_function` with the argument `max_complexity`, which speeds up lambdification (see {doc}`/usage/faster-lambdify`).

In [ ]:
import pickle

import qrules
from ampform.helicity import HelicityModel

from tensorwaves.function.sympy import create_parametrized_function

reaction = qrules.io.load("transitions.json")
with open("helicity_model.pickle", "rb") as stream:
    model: HelicityModel = pickle.load(stream)
with open("data.pickle", "rb") as stream:
    data = pickle.load(stream)
with open("phsp.pickle", "rb") as stream:
    phsp = pickle.load(stream)

function = create_parametrized_function(
    expression=model.expression.doit(),
    parameters=model.parameter_defaults,
    backend="jax",
    max_complexity=100,
)

### 3.1 Define estimator

To perform a fit, you need to define an {class}`.Estimator`. This is a measure for the discrepancy between the {class}`.ParametrizedFunction` and the data distribution to which you fit it. In PWA, we usually use an **unbinned negative log likelihood estimator** ({class}`.UnbinnedNLL`).

Generally, the {class}`.ParametrizedFunction` is not normalized with regards to the data sample, while a log likelihood estimator requires a normalized function. This is where the {ref}`phase space data <usage/ampform:2.1 Generate phase space sample>` comes into play again: the {class}`.ParametrizedFunction` is evaluated over the phase space data, so that its output can be used as a normalization factor.

```{margin}
If you want to correct for the efficiency of the detector, you should use a *detector-reconstructed* phase space sample.
```

In [ ]:
from tensorwaves.estimator import UnbinnedNLL

estimator = UnbinnedNLL(
    function,
    data=data,
    phsp=phsp,
    backend="jax",
)

Note that the {class}`.UnbinnedNLL` can be expressed with different backends, because it uses statistical operations like `log` and `mean`. Here, we use {func}`jax <jax.jit>`, which turns out to be the fastest backend for this model.

### 3.2 Optimize intensity model

#### Specify fit parameters

Starting the fit itself is quite simple: just create an {mod}`.optimizer` instance of your choice, here [Minuit2](https://root.cern.ch/doc/master/Minuit2Page.html), and call its {meth}`~.Optimizer.optimize` method to start the fitting process. The {meth}`~.Optimizer.optimize` method requires a mapping of parameter names to their initial values. **Only the parameters listed in the mapping are optimized.**

Let's first select a few of parameters and give them a small offset with regard to their original value. We give this offset to make the fit more interesting―after all, we are fitting to a data sample that was generated with this very same {class}`.ParametrizedFunction`.

In [ ]:
original_parameters = function.parameters
initial_parameters = {
    R"C_{J/\psi(1S) \to f_{0}(1500)_{0} \gamma_{+1}; f_{0}(1500) \to \pi^{0}_{0} \pi^{0}_{0}}": 1.0
    + 0.0j,
    "Gamma_f(0)(980)": 0.15,
    "Gamma_f(0)(1500)": 0.2,
    "m_f(0)(1710)": 1.78,
}

Recall that a {class}`.ParametrizedFunction` object computes the intensity for a certain `.DataSample`. This can be seen now nicely when we use these intensities as weights on the phase space sample and plot it together with the original data sample. Here, we look at the invariant mass distribution projection of the final states `1` and `2`, which, {ref}`as we saw before <usage/ampform:2.3 Visualize kinematic variables>`, is the final state particle pair $\pi^0\pi^0$.

Don't forget to use {meth}`~.ParametrizedFunction.update_parameters` first!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

reaction_info = model.reaction_info
resonances = sorted(
    reaction_info.get_intermediate_particles(),
    key=lambda p: p.mass,
)

evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [cm.rainbow(x) for x in evenly_spaced_interval]


def indicate_masses():
    plt.xlabel("$m$ [GeV]")
    for color, resonance in zip(colors, resonances):
        plt.gca().axvline(
            x=resonance.mass,
            linestyle="dotted",
            label=resonance.name,
            color=color,
        )


def compare_model(
    variable_name,
    data,
    phsp,
    function,
    bins=100,
):
    intensities = function(phsp)
    _, ax = plt.subplots(figsize=(9, 4))
    data_1d = data[variable_name]
    ax = plt.gca()
    ax.hist(
        data_1d,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    phsp_1d = phsp[variable_name]
    ax.hist(
        phsp_1d,
        weights=np.array(intensities),
        bins=bins,
        histtype="step",
        color="red",
        label="initial fit model",
        density=True,
    )
    indicate_masses()
    ax.legend()

In [ ]:
function.update_parameters(initial_parameters)
compare_model("m_12", data, phsp, function)

#### Custom callbacks

The {class}`.Minuit2` class allows one to insert certain {mod}`~tensorwaves.optimizer.callbacks`. Callbacks are used to insert behavior into the {meth}`.Optimizer.optimize` method. The {mod}`~tensorwaves.optimizer.callbacks` module provides several handy callback classes, but it's also possible to define custom callbacks into the {class}`.Optimizer` by defining a custom {class}`.Callback` class. Here's one that live updates a plot of the latest fit model!

In [ ]:
from IPython.display import clear_output

from tensorwaves.optimizer.callbacks import Callback


class PyplotCallback(Callback):
    def __init__(self, variable="m_12", step_size=10):
        self.__variable = variable
        self.__step_size = step_size
        self.__fig = None
        self.__ax = None
        self.__latest_parameters = {}

    def on_optimize_start(self, logs):
        if STATIC_WEB_PAGE:
            return
        self.__fig, self.__ax = plt.subplots(1, figsize=(8, 5))

    def on_optimize_end(self, logs):
        if STATIC_WEB_PAGE:
            return
        self.update_plot(nbins=200)
        self.__ax = None
        self.__fig = None

    def on_iteration_end(self, iteration, logs=None):
        pass

    def on_function_call_end(self, function_call, logs):
        if STATIC_WEB_PAGE:
            return
        self.__latest_parameters = logs["parameters"]
        if function_call % self.__step_size != 0:
            return
        if function_call > 75:
            return
        self.update_plot(nbins=80)
        clear_output(wait=True)
        display(plt.gcf())

    def update_plot(self, nbins: int):
        if self.__fig is None or self.__ax is None:
            self.__fig, self.__ax = plt.subplots(1, figsize=(8, 5))
        function.update_parameters(self.__latest_parameters)
        intensities = function(phsp)
        self.__ax.cla()
        data_1d = data[self.__variable]
        self.__ax.hist(
            data_1d, bins=nbins, alpha=0.5, label="data", density=True
        )
        phsp_1d = phsp[self.__variable]
        self.__ax.hist(
            phsp_1d,
            weights=np.array(intensities),
            bins=nbins,
            histtype="step",
            color="red",
            label="fit model",
            density=True,
        )
        self.__ax.set_xlim((0.25, 2.5))
        self.__ax.set_ylim((0, 1.9))
        indicate_masses()
        plt.gcf().legend()

In the fit example below, we want to use several callbacks together. To do so, we 'stack' them together with {class}`.CallbackList`:

In [ ]:
from tensorwaves.optimizer.callbacks import (
    CallbackList,
    CSVSummary,
    TFSummary,
    YAMLSummary,
)

callbacks = CallbackList(
    [
        CSVSummary("fit_traceback_minuit.csv"),
        PyplotCallback(),  # comment this line for raw fit performance
        TFSummary(),
        YAMLSummary("current_fit_result.yaml"),
    ]
)

#### Perform fit

Finally, we create an {class}`.Optimizer` to {meth}`~.Minuit2.optimize` the parameters in the {class}`.ParametrizedFunction` (which is embedded in the {class}`.Estimator`). Here, we choose the {class}`.Minuit2` optimizer, which is the most common optimizer in high-energy physics. Since we constructed the {class}`.UnbinnedNLL` with {obj}`jax <jax.jit>`, can an optimize the model with analytic gradient over the {class}`.ParametrizedBackendFunction`:

```{margin}
The computation time depends on the complexity of the model, the number of data events, the size of the phase space sample, and the number of free parameters. This model is rather small and has but a few free parameters, so the optimization shouldn't take more than a minute.

The custom callback example slows down the optimization, but you can remove it to get the raw fit performance.
```

In [ ]:
from tensorwaves.optimizer import Minuit2

minuit2 = Minuit2(
    callback=callbacks,
    use_analytic_gradient=False,
)
fit_result = minuit2.optimize(estimator, initial_parameters)
fit_result

In [ ]:
assert fit_result.minimum_valid

As can be seen, the values of the optimized parameters in the {class}`.FitResult` are again comparable to the original parameter values.

##### Covariance matrix

Each of the {mod}`.optimizer`s offer more specific information about the fit result. This information can be accessed with {attr}`.FitResult.specifics`. A common example would be to get the {attr}`~iminuit.Minuit.covariance` matrix:

In [ ]:
covariance_matrix = fit_result.specifics.covariance
covariance_matrix.correlation()

##### AIC and BIC

As an example, here is how to compute the [AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion#Definition) and [BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion#Definition) from this {obj}`.FitResult`:

In [ ]:
n_real_par = fit_result.count_number_of_parameters(complex_twice=True)
n_events = len(list(data.values())[0])
log_likelihood = -fit_result.estimator_value

bic = n_real_par * np.log(n_events) - 2 * log_likelihood
aic = 2 * n_real_par - 2 * log_likelihood

In [ ]:
print("AIC:", aic)
print("BIC:", bic)

In [ ]:
optimized_parameters = fit_result.parameter_values
for p in optimized_parameters:
    print(p)
    print(f"  initial:   {initial_parameters[p]:.3}")
    print(f"  optimized: {optimized_parameters[p]:.3}")
    print(f"  original:  {original_parameters[p]:.3}")

:::{seealso}

{ref}`usage/ampform:SciPy optimizer`

:::

### 3.3 Export and import

In {ref}`usage/ampform:3.2 Optimize intensity model`, we initialized {obj}`.Minuit2` with some callbacks that are {class}`.Loadable`. Such callback classes offer the possibility to {meth}`~.Loadable.load_latest_parameters`, so you can pick up the optimize process in case it crashes or if you pause it. Loading the latest parameters goes as follows:

In [ ]:
latest_parameters = YAMLSummary.load_latest_parameters(
    "current_fit_result.yaml"
)
latest_parameters

To restart the fit with the latest parameters, simply rerun as before.

In [ ]:
minuit2 = Minuit2()
fit_result = minuit2.optimize(estimator, latest_parameters)
fit_result

In [ ]:
assert fit_result.minimum_valid

Lo and behold: the parameters were already optimized, so the fit converged faster!

### 3.4 Visualize

#### Plot optimized model

Using the same method as above, we renew the parameters of the {class}`.ParametrizedFunction` and plot it again over the phase space sample.

In [ ]:
function.update_parameters(latest_parameters)
compare_model("m_12", data, phsp, function)

#### Intensity components

Notice that the {class}`~ampform.helicity.HelicityModel` contains {attr}`~ampform.helicity.HelicityModel.components` attribute. This is {obj}`dict` of component names to {class}`sympy.Expr <sympy.core.expr.Expr>`s helps us to identify amplitudes and intensities in the total amplitude.

In [ ]:
sorted(model.components)

In [ ]:
model.components[
    R"A_{J/\psi(1S)_{+1} \to f_{0}(1370)_{0} \gamma_{+1}; f_{0}(1370)_{0} \to"
    R" \pi^{0}_{0} \pi^{0}_{0}}"
].subs(model.parameter_defaults).doit()

Just like in {ref}`usage/ampform:2.2 Generate intensity-based sample`, these _intensity components_ can each be expressed in a computational backend. This can be done with the method {meth}`~ampform.helicity.HelicityModel.sum_components` from the original model and creating a new {class}`.ParametrizedBackendFunction` from that expression with {func}`.create_parametrized_function`. After that we, update the parameters with the optimized parameter values we found in {ref}`usage/ampform:Perform fit`. The two components in the example below should be the same:

In [ ]:
added_components = model.sum_components(
    components=[
        R"A_{J/\psi(1S)_{+1} \to f_{0}(500)_{0} \gamma_{+1}; f_{0}(500)_{0}"
        R" \to \pi^{0}_{0} \pi^{0}_{0}}",
        R"A_{J/\psi(1S)_{+1} \to f_{0}(980)_{0} \gamma_{+1}; f_{0}(980)_{0}"
        R" \to \pi^{0}_{0} \pi^{0}_{0}}",
        R"A_{J/\psi(1S)_{+1} \to f_{0}(1370)_{0} \gamma_{+1}; f_{0}(1370)_{0}"
        R" \to \pi^{0}_{0} \pi^{0}_{0}}",
        R"A_{J/\psi(1S)_{+1} \to f_{0}(1500)_{0} \gamma_{+1}; f_{0}(1500)_{0}"
        R" \to \pi^{0}_{0} \pi^{0}_{0}}",
        R"A_{J/\psi(1S)_{+1} \to f_{0}(1710)_{0} \gamma_{+1}; f_{0}(1710)_{0}"
        R" \to \pi^{0}_{0} \pi^{0}_{0}}",
    ]
)
from_amplitudes = create_parametrized_function(
    expression=added_components.doit(),
    parameters=model.parameter_defaults,
    backend="numpy",
)
from_amplitudes.update_parameters(latest_parameters)

In [ ]:
added_components = model.sum_components(
    components=[R"I_{J/\psi(1S)_{+1} \to \gamma_{+1} \pi^{0}_{0} \pi^{0}_{0}}"]
)
from_intensity = create_parametrized_function(
    expression=added_components.doit(),
    parameters=model.parameter_defaults,
    backend="numpy",
)
from_intensity.update_parameters(latest_parameters)

In [ ]:
import numpy as np

difference = np.average(from_amplitudes(phsp) - from_intensity(phsp))
assert np.round(difference, decimals=15) == 0.0

The result is a {class}`.ParametrizedBackendFunction` that can be plotted just like in {ref}`usage/ampform:Plot optimized model`:

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 5))
bins = 150
ax.hist(
    phsp["m_12"],
    weights=np.array(function(phsp)),
    bins=bins,
    alpha=0.2,
    label="full intensity",
)
ax.hist(
    phsp["m_12"],
    weights=np.array(from_intensity(phsp)),
    bins=bins,
    histtype="step",
    label=R"$J/\psi(1S)_{-1} \to \gamma_{-1} \pi^0 \pi^0$",
)
ax.set_xlim(0.25, 2.5)
ax.set_xlabel(R"$m_{\pi^0\pi^0}$ [GeV]")
ax.set_yticks([])
plt.legend()
plt.show()

Or generically, so that we can stack all the sub-intensities:

In [ ]:
import logging

from tensorwaves.data import generate_data
from tensorwaves.data.transform import SympyDataTransformer

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)
intensity_components = [
    create_parametrized_function(
        expression=model.sum_components([c]).doit(),
        parameters=model.parameter_defaults,
        backend="numpy",
    )
    for c in model.components
    if c.startswith("I")
]
initial_state_mass = reaction_info.initial_state[-1].mass
final_state_masses = {i: p.mass for i, p in reaction_info.final_state.items()}

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="numpy"
)
masses = []
for component in intensity_components:
    sub_events = generate_data(
        size=5_000,
        initial_state_mass=initial_state_mass,
        final_state_masses=final_state_masses,
        data_transformer=helicity_transformer,
        intensity=component,
    )
    sub_dataset = helicity_transformer(sub_events)
    masses.append(sub_dataset["m_12"])

fig, ax = plt.subplots(1, figsize=(8, 5))
plt.hist(
    masses,
    bins=100,
    stacked=True,
    alpha=0.6,
)
ax.set_xlim(0.25, 2.5)
ax.set_xlabel(R"$m_{\pi^0\pi^0}$ [GeV]")
ax.set_yticks([])
ax.legend(
    labels=[f"${c[3:-1]}$" for c in model.components if c.startswith("I")]
)

#### Analyze optimization process

Note that {ref}`in Step 3.2 <usage/ampform:3.2 Optimize intensity model>`, we initialized {class}`.Minuit2` with a {class}`.TFSummary` callback as well. Its output files provide a nice, interactive representation of the fit process and can be viewed with [TensorBoard](https://www.tensorflow.org/tensorboard/get_started) as follows:

````{tabbed} Terminal
```bash
tensorboard --logdir logs
```
````

````{tabbed} Python
```python
import tensorboard as tb

tb.notebook.list()  # View open TensorBoard instances
tb.notebook.start(args_string="--logdir logs")
```
See more info [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks#tensorboard_in_notebooks)
````

````{tabbed} Jupyter notebook
```ipython
%load_ext tensorboard
%tensorboard --logdir logs
```
See more info [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks#tensorboard_in_notebooks)
````

An alternative would be to use the output of the {class}`.CSVSummary` callback. Here's an example:

In [ ]:
import pandas as pd
import sympy as sp

converters = {p: lambda s: complex(s).real for p in initial_parameters}
fit_traceback = pd.read_csv("fit_traceback_minuit.csv", converters=converters)
fig, (ax1, ax2) = plt.subplots(
    2, figsize=(7, 8), gridspec_kw={"height_ratios": [1, 1.8]}
)
fit_traceback.plot("function_call", "estimator_value", ax=ax1)
fit_traceback.plot("function_call", sorted(initial_parameters), ax=ax2)
fig.suptitle("Minuit optimizer process", fontsize=16)
ax1.set_title("Negative log likelihood")
ax2.set_title("Parameter values")
ax1.set_xlabel("function call")
ax2.set_xlabel("function call")
fig.tight_layout()
ax1.legend().remove()
legend_texts = ax2.legend().get_texts()
for text in legend_texts:
    latex = f"${sp.latex(sp.Symbol(text.get_text()))}$"
    latex = latex.replace("\\\\", "\\")
    if latex[2] == "C":
        latex = fR"\left|{latex}\right|"
    text.set_text(latex)
for line in ax2.get_lines():
    label = line.get_label()
    color = line.get_color()
    ax2.axhline(
        y=complex(original_parameters[label]).real,
        color=color,
        alpha=0.5,
        linestyle="dotted",
    )

### SciPy optimizer

As an alternative to the {class}`.Minuit2` optimizer, you can use the {class}`.ScipyMinimizer`. As opposed to Minuit, this optimizer does not compute errors. See {doc}`scipy:tutorial/optimize` and {mod}`scipy.optimize` for more info.

In [ ]:
from tensorwaves.optimizer.scipy import ScipyMinimizer

scipy = ScipyMinimizer(
    callback=CSVSummary(
        "fit_traceback_scipy.csv",
        function_call_step_size=1,
        iteration_step_size=1,
    ),
    use_analytic_gradient=False,
)
fit_result = scipy.optimize(estimator, initial_parameters)
fit_result

In [ ]:
converters = {p: lambda s: complex(s).real for p in initial_parameters}
fit_traceback = pd.read_csv("fit_traceback_scipy.csv", converters=converters)
fig, (ax1, ax2) = plt.subplots(
    2, figsize=(7, 8), gridspec_kw={"height_ratios": [1, 1.8]}
)
fit_traceback.plot("function_call", "estimator_value", ax=ax1)
fit_traceback.plot("function_call", sorted(initial_parameters), ax=ax2)
fig.suptitle("SciPy optimizer process", fontsize=16)
ax1.set_title("Negative log likelihood")
ax2.set_title("Parameter values")
ax1.set_xlabel("function call")
ax2.set_xlabel("function call")
fig.tight_layout()
ax1.legend().remove()
legend_texts = ax2.legend().get_texts()
for text in legend_texts:
    latex = f"${sp.latex(sp.Symbol(text.get_text()))}$"
    latex = latex.replace("\\\\", "\\")
    if latex[2] == "C":
        latex = fR"\left|{latex}\right|"
    text.set_text(latex)
for line in ax2.get_lines():
    label = line.get_label()
    color = line.get_color()
    ax2.axhline(
        y=complex(original_parameters[label]).real,
        color=color,
        alpha=0.5,
        linestyle="dotted",
    )